<a href="https://colab.research.google.com/github/mmorales20/CS470/blob/master/11_11_DataConverterMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

>[Packages](#scrollTo=tkKZZHqhNzuN)

>[Reading in CSV Files](#scrollTo=q7zSBFfcPXG_)

>[Categorical Data finder](#scrollTo=LX7CdYrqTwe8)

>[Numeric Data Finder](#scrollTo=5yOTozz4UfvX)

>[SDT Changes](#scrollTo=BeheiUGPVn75)

>[ETC](#scrollTo=wSVYcrMuWl-U)

>[Runner](#scrollTo=zc5tcEDYWyw5)



# Packages


In [0]:
#Download packages
import csv
import pandas as pd
import re
import xml.etree.ElementTree as ET

#Download package for File Explorer pop up
from tkinter import filedialog
from tkinter import *

# Reading in CSV Files

In [0]:
#Display a file explorer and allow user to select a CSV file that contains the desire data
def getFile(sdtFile):
    root = Tk()
    root.filename =  filedialog.askopenfilename(initialdir = "/",title = "Select file",filetypes = (("text files","*.csv"),("all files","*.*")))
    changePath(root.filename,sdtFile)
    root.destroy()
    return(root.filename)

In [0]:
#Change the path based on the file selected by the user
def changePath(path,sdtFile):
    tree = ET.parse(sdtFile)
    root = tree.getroot() 
    
    for elem in root.findall('.//Datasources/Datasource/Host'):
        elem.text=path
    
    for elem in root.findall('.//Datasources/Datasource/Name'):
        elem.text=path
        
    tree.write(sdtFile,encoding="utf-8", xml_declaration=True)

In [0]:
#Read in each column of the CSV file that was selected
def readCSV(csvName):
    df = pd.read_csv(csvName)
    print("reading csv done")
    return df

# Categorical Data finder

In [0]:
#Add each categoircal data column to a list
def getCat(df,colorName):
    x=0
    for name in df.columns:
        if df.dtypes[x] == 'object':
             if len(df[name].astype('category').cat.categories.tolist()) >1 and len(df[name].astype('category').cat.categories.tolist()) <101 and colorName=="": 
                colorName=df.columns[x]
        x= x+1
    return colorName

In [0]:
#Within each column save each unique attribute to a list
def getCatCol(df,colorName):
    return df[colorName].astype('category').cat.categories.tolist()

# Numeric Data Finder

In [0]:
#Find numeric data and add the column names to a list
def typeDefiner(df,catFound,sdtFile):
    colList = df.columns
    colList
    x = 0
    colNames = []
    for name in colList:
        if df.dtypes[x] == 'object':# or df.dtypes[x] == 'int64':
            pass
        else:
            colNames.append(colList[x])
                
        x = x + 1
    
    #Finds number of numeric inputs in case of no categorical data
    file = ET.parse(sdtFile)
    root = file.getroot()
    inputCount=0
    for elem in root.findall('.//InputFields'):
        for inputs in elem.findall('.//InputField'):
            inputCount+=1
    
    #Resets part of color section and input if there is no option for color
    #NOT WORKING DO NOT ATTEMPT AT THIS MOMENT
    if not catFound and inputCount>len(colNames):
        resetToDefault(sdtFile)
    print("input count is "+str(catFound) +str(inputCount))
    print("objects removed")
    print(colNames)
    return colNames

# SDT Changes

In [0]:
#HAVING ERRORS DO NOT USE YET
def toNumeric(sdtFile):

    #import xml.etree.ElementTree as ET
    #CLEARS COLOR OUT NOW WE CAN INSERT ANYTHING
    file = ET.parse(sdtFile)
    root = file.getroot()
    for elem in root.findall('.//Glyph/Color/RGB/Function[@type="Text Field To Value"]/KeyValuePairs'):
        elem.clear()

    for parent in root.findall('.//KeyValuePairs/..'):
        for element in parent.findall('KeyValuePairs'):
            parent.remove(element)
   #Try to write here
   # for elem in root.findall('.//Color/RGB/..):
      #  check_req_elems = elem.find('.//Function[@type="Text Field To Value"]')
        #check_req_elems.type = "Linear Interpolation"
       # check_req_elems.set("type","Linear Interpolation")
       # attrib = {'type': 'BoundField'}
       # minmax = ET.SubElement(check_req_elems, 'MinMax', attrib)
       # minmax.text= "\n\t\t\t\t\t\t\t"
       # min = ET.SubElement(minmax, 'Min')
       # min.text = '0'
       # min.tail="\n\t\t\t\t\t\t"
       # minmax.tail="\n\t\t\t\t\t"
    file.write(sdtFile)
    print("color fixed")

In [0]:
#Accepted Params: list of column names, the .SDT file, column name, and boolean
def XMLParser(colNames,sdtFile,colorName,catFound):
    print(colNames)

    tree = ET.parse(sdtFile)
    root = tree.getroot()
    count=0;
    print(len(colNames))
    for elem in root.iter('InputField'):
        print(count)
        if elem.get('name')!="color":
            elem.set('field', colNames[count])
            count+=1
        elif not catFound:
            elem.set('field', colNames[count])
            count+=1
        else:
            elem.set('field', colorName)
        
    tree.write(sdtFile)
    print("XML updated")

In [0]:
#
def setupColor(colorCol,sdtFile):
    file = ET.parse(sdtFile)
    root = file.getroot()
    x=0
    if(colorCol==None):#and then potentially add a binding field
        toNumeric(sdtFile)
        
    else:
        for elem in root.findall('.//Color/RGB/Function[@type="Text Field To Value"]/KeyValuePairs'):
            for element in elem.findall('KeyValuePair/Key'):
                element.text=colorCol[x]
                x=x+1

            if(len(colorCol)>2):
                while x<len(colorCol):
                    for elem in root.findall('.//Color/RGB/Function[@type="Text Field To Value"]'):
                        pairs = elem.find('.//KeyValuePairs')
                #fixes spacing so new pair will be at right column/indent area
                        for child in pairs.findall('.//KeyValuePair'):
                            child.tail="\n\t\t\t\t\t\t\t"
                        keyVal =ET.SubElement(pairs, 'KeyValuePair')
                        keyVal.text="\n\t\t\t\t\t\t\t\t"
                        keyVal.tail="\n\t\t\t\t\t\t"
                        keyValP= ET.SubElement(keyVal, 'Key')
                        keyValP.text=colorCol[x]
                        keyValP.tail="\n\t\t\t\t\t\t\t\t"
                        key = ET.SubElement(keyVal, 'Value')
                    
                        key.tail="\n\t\t\t\t\t\t\t"
                        x=x+1
    
                        x=0

                        count=0
                        colorList=[] 
                        while count<len(colorCol):
                            randomCol(colorList)
                            count=count+1


                        for elem in root.findall('.//Color/RGB/Function[@type="Text Field To Value"]/KeyValuePairs'):
                            for element in elem.findall('KeyValuePair/Value'):
                                element.text=colorList[x]
                                print(colorList[x])
                                x=x+1
                
                
    file.write(sdtFile,encoding="utf-8", xml_declaration=True)


In [0]:
#
def moveToNewFile(sdtFile):
    file = ET.parse(sdtFile)
    file.write('emptySDT1.sdt',encoding="utf-8", xml_declaration=True)
    
    return 'emptySDT1.sdt'

In [0]:
def resetToDefault(sdtFile):
    file = ET.parse(sdtFile)
    root = file.getroot()
    print("resetting")
    for parent in root.findall('.//Glyph/Color/RGB/..'):
        for element in parent.findall('Binding[@fieldId="color"]'):
            parent.remove(element)
    
   # for parent in root.findall('.//InputFields/..'):
    #    for element in parent.findall('.//InputField[@name="color"]/'):
     #       parent.remove(element)
            
            
    #for elem in root.findall('.//Color/RGB/Function[@type="Text Field To Value"]/KeyValuePairs'):
     #   elem.clear()

    #for parent in root.findall('.//KeyValuePairs/..'):
    #    for element in parent.findall('KeyValuePairs'):
    #        parent.remove(element)
    file.write(sdtFile,encoding="utf-8", xml_declaration=True)

# ETC


In [0]:
#Random Color Generator
def randomCol(colorList):
    import random
    r = random.random()*255
    r=int(r)
    g = random.random()*255
    g=int(g)
    b = random.random()*255
    b=int(b)
    color=str(r)+","+str(g)+","+str(b)
    colorList.append(color)

#Runner

In [0]:
def runAll(sdtFile):
    colorCol= None
    colorName=""
    catFound=False
    csv = getFile(sdtFile)
    df = readCSV(csv)
    colorName=getCat(df,colorName)
    print(colorName)
    if(colorName!=""):
        catFound=True
        colorCol=getCatCol(df,colorName)
  
    print(catFound)
    sdtFile=moveToNewFile(sdtFile)
    colNames = typeDefiner(df,catFound,sdtFile)
    #colNames.append(colorName)
    XMLParser(colNames, sdtFile,colorName,catFound)
    setupColor(colorCol, sdtFile)
   # csv = getFile()
#readCSV(csv)

In [0]:
runAll('emptySDT.sdt')

TclError: ignored